In [1]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster, HeatMap
import numpy as np
from sklearn.cluster import DBSCAN
import os

# Define color function based on pH value ranges for sea water
def get_ph_color(ph):
    if pd.isna(ph):
        return 'gray', 'No Data'
    elif ph <= 4:
        return '#FF0000', 'Highly Acidic'      # red
    elif ph < 6.5:
        return '#FFFF00', 'Moderately Acidic'  # yellow
    elif ph <= 8.5:
        return '#00FF00', 'Acceptable'         # green
    elif ph < 10:
        return '#00BFFF', 'Moderately Alkaline'  # sky blue
    else:
        return '#8A2BE2', 'Highly Alkaline'     # violet


# Define color function for Dissolved Oxygen (DO) for sea water
def get_do_color(do):
    if pd.isna(do):
        return 'gray', 'No Data'
    elif do >= 5:
        return '#00FF00', 'Acceptable'     # green - DO >= 5mg/L
    elif do >= 3:
        return '#FFFF00', 'Moderate'       # yellow - 3 <= DO < 5 mg/L
    else:
        return '#FF0000', 'Low Oxygen'     # red - DO < 3mg/L

# Define color function for Biochemical Oxygen Demand (BOD) for sea water
def get_bod_color(bod):
    if pd.isna(bod):
        return 'gray', 'No Data'
    elif bod <= 2:
        return '#00FF00', 'Acceptable'     # green - BOD <= 2mg/L (stricter for sea water)
    elif bod <= 5:
        return '#FFFF00', 'Moderate'       # yellow - 2 < BOD <= 5mg/L
    else:
        return '#FF0000', 'High BOD'       # red - BOD > 5mg/L

# Assign color by water body type
def get_body_color(water_body_type):
    color_map = {
        'river': '#4682B4',
        'sea': '#4169E1',
        'drain': '#696969',
        'marine': '#008080',
        'lake': '#1E90FF',
        'creek': '#20B2AA',
        'canal': '#2F4F4F',
    }
    for key in color_map:
        if key in str(water_body_type).lower():
            return color_map[key]
    return '#AAAAAA'

def load_data_for_year(year):
    file_path = f'seas_with_location_{year}.csv'
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        return None
    df = pd.read_csv(file_path)

    numeric_cols = [
        'TEMPERATURE_Min', 'TEMPERATURE_Max', 'DO_Min', 'DO_MAX',
        'pH_Min', 'PH_Max', 'CONDUCTIVITY_Min', 'CONDUCTIVITY_Max',
        'BOD_Min', 'BOD_Max', 'NITRATE_NITRATE_Min', 'NITRATE_NITRATE_Max',
        'FECALCOLIFORM_Min', 'FECALCOLIFORM_Max', 'TOTALCOLIFORM_Min',
        'TOTALCOLIFORM_Max', 'Latitude', 'Longitude'
    ]
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    df = df.dropna(subset=['Latitude', 'Longitude'])
    return df

def create_sea_water_map(data, year, parameter='pH'):
    """Create a map with sea/marine water bodies colored according to selected parameter values"""
    india_map = folium.Map(location=[22.5, 79.5], zoom_start=5.4, tiles="CartoDB positron")
    
    folium.TileLayer('openstreetmap').add_to(india_map)
    folium.TileLayer(
        tiles='https://stamen-tiles.a.ssl.fastly.net/terrain/{z}/{x}/{y}.png',
        attr='Map tiles by Stamen Design, CC BY 3.0 — Map data © OpenStreetMap contributors',
        name='Stamen Terrain',
        overlay=False,
        control=True
    ).add_to(india_map)

    title_html = f'''<h3 align="center" style="font-size:20px"><b>India Sea/Marine Water {parameter} Map - {year}</b></h3>'''
    india_map.get_root().html.add_child(folium.Element(title_html))

    marker_cluster = MarkerCluster(disableClusteringAtZoom=7).add_to(india_map)
    
    coords = data[['Latitude', 'Longitude']].values
    clustering = DBSCAN(eps=0.2, min_samples=1).fit(coords)
    data['cluster'] = clustering.labels_

    # Set parameter column and color function based on selected parameter
    if parameter == 'pH':
        param_col = 'PH_Max'
        color_func = get_ph_color
    elif parameter == 'DO':
        param_col = 'DO_MAX'
        color_func = get_do_color
    elif parameter == 'BOD':
        param_col = 'BOD_Max'
        color_func = get_bod_color

    # Group by clusters and add polylines
    for cluster_id, group in data.groupby('cluster'):
        if len(group) >= 2:
            avg_val = group[param_col].mean()
            color, category = color_func(avg_val)
            coords = group.sort_values(['Latitude', 'Longitude'])[['Latitude', 'Longitude']].values.tolist()
            folium.PolyLine(
                locations=coords,
                color=color,
                weight=4,
                opacity=0.7,
                tooltip=f"Segment {cluster_id} | Avg {parameter}: {avg_val:.2f} ({category})"
            ).add_to(india_map)

    # Add markers for each station
    for idx, row in data.iterrows():
        param_value = row[param_col]
        color, category = color_func(param_value)
        body_color = get_body_color(row.get('WATER_BODY_TYPE', ''))

        popup_html = f"""
        <div style='width: 250px; overflow-wrap: break-word; font-size: 13px;'>
            <b>Station:</b> {row.get('STATION CODE', 'N/A')}<br>
            <b>State:</b> {row.get('STATE', 'N/A')}<br>
            <b>Water Body:</b> {row.get('WATER_BODY_TYPE', 'N/A')}<br>
            <b>{parameter}:</b> {f"{param_value:.2f}" if not pd.isna(param_value) else 'N/A'}<br>
            <b>Category:</b> {category}
        </div>
        """
        try:
            folium.CircleMarker(
                location=[row['Latitude'], row['Longitude']],
                radius=6,
                color=color,
                fill=True,
                fill_opacity=0.8,
                popup=folium.Popup(popup_html, max_width=300)
            ).add_to(marker_cluster)
        except:
            print(f"Skipped invalid coordinates for station: {row.get('STATION CODE')}")

    # Add appropriate legend based on parameter
    if parameter == 'pH':
        legend_html = '''
        <div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000; background: white;
                    border:2px solid gray; border-radius:5px; padding: 10px; font-size:13px;">
            <h4 style="margin: 0 0 5px;">pH Categories (Sea Water)</h4>
            <h4>pH Levels</h4>
        <div><i style="background: #FF0000; width: 15px; height: 15px; display: inline-block;"></i> pH ≤ 4 (Highly Acidic)</div>
        <div><i style="background: #FFFF00; width: 15px; height: 15px; display: inline-block;"></i> 4 < pH < 6.5 (Moderately Acidic)</div>
        <div><i style="background: #00FF00; width: 15px; height: 15px; display: inline-block;"></i> 6.5 ≤ pH ≤ 8.5 (Acceptable)</div>
        <div><i style="background: #00BFFF; width: 15px; height: 15px; display: inline-block;"></i> 8.5 < pH < 10 (Moderately Alkaline)</div>
        <div><i style="background: #8A2BE2; width: 15px; height: 15px; display: inline-block;"></i> pH ≥ 10 (Highly Alkaline)</div>
        
            <hr style="margin: 5px 0;">
            <b>Line Colors:</b> Cluster Avg pH<br>
            <b>Dot Colors:</b> Station pH
        </div>
        '''
    elif parameter == 'DO':
        legend_html = '''
        <div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000; background: white;
                    border:2px solid gray; border-radius:5px; padding: 10px; font-size:13px;">
            <h4 style="margin: 0 0 5px;">Dissolved Oxygen (DO) Categories</h4>
            <div><i style="background:#00FF00;width:15px;height:15px;display:inline-block;"></i> ≥ 5 mg/L (Acceptable)</div>
            <div><i style="background:#FFFF00;width:15px;height:15px;display:inline-block;"></i> 3-5 mg/L (Moderate)</div>
            <div><i style="background:#FF0000;width:15px;height:15px;display:inline-block;"></i> < 3 mg/L (Low Oxygen)</div>
            <div><i style="background:gray;width:15px;height:15px;display:inline-block;"></i> No Data</div>
            <hr style="margin: 5px 0;">
            <b>Line Colors:</b> Cluster Avg DO<br>
            <b>Dot Colors:</b> Station DO
        </div>
        '''
    elif parameter == 'BOD':
        legend_html = '''
        <div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000; background: white;
                    border:2px solid gray; border-radius:5px; padding: 10px; font-size:13px;">
            <h4 style="margin: 0 0 5px;">BOD Categories (Sea Water)</h4>
            <div><i style="background:#00FF00;width:15px;height:15px;display:inline-block;"></i> ≤ 2 mg/L (Acceptable)</div>
            <div><i style="background:#FFFF00;width:15px;height:15px;display:inline-block;"></i> 2-5 mg/L (Moderate)</div>
            <div><i style="background:#FF0000;width:15px;height:15px;display:inline-block;"></i> > 5 mg/L (High BOD)</div>
            <div><i style="background:gray;width:15px;height:15px;display:inline-block;"></i> No Data</div>
            <hr style="margin: 5px 0;">
            <b>Line Colors:</b> Cluster Avg BOD<br>
            <b>Dot Colors:</b> Station BOD
        </div>
        '''
    
    india_map.get_root().html.add_child(folium.Element(legend_html))
    return india_map

def create_parameter_year_selector_app():
    """Create an interactive application with year and parameter selector for sea water quality"""
    # Create output directories for maps
    if not os.path.exists('maps'):
        os.makedirs('maps')
    if not os.path.exists('maps/sea_pH'):
        os.makedirs('maps/sea_pH')
    if not os.path.exists('maps/sea_DO'):
        os.makedirs('maps/sea_DO')
    if not os.path.exists('maps/sea_BOD'):
        os.makedirs('maps/sea_BOD')
    
    # Create maps for each year and parameter and save them
    parameters = ['pH', 'DO', 'BOD']
    for year in range(2012, 2024):
        data = load_data_for_year(year)
        if data is not None:
            for param in parameters:
                map_obj = create_sea_water_map(data, year, parameter=param)
                map_obj.save(f'maps/sea_{param}/india_sea_{param.lower()}_map_{year}.html')
                print(f"Generated sea water {param} map for {year}")
    
    # Create HTML file with year and parameter selectors
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>India Sea Water Quality Map</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                margin: 0;
                padding: 0;
            }
            .container {
                display: flex;
                flex-direction: column;
                height: 100vh;
            }
            .header {
                background-color: #f0f0f0;
                padding: 10px;
                text-align: center;
            }
            .content {
                display: flex;
                flex: 1;
            }
            .map-container {
                flex: 1;
                position: relative;
            }
            iframe {
                width: 100%;
                height: 100%;
                border: none;
            }
            .selectors {
                padding: 20px;
                display: flex;
                justify-content: center;
                align-items: center;
                gap: 20px;
                flex-wrap: wrap;
            }
            .parameter-selector {
                display: flex;
                flex-direction: column;
                align-items: center;
            }
            .year-selector {
                display: flex;
                justify-content: center;
                gap: 10px;
                flex-wrap: wrap;
            }
            select {
                padding: 8px;
                border-radius: 4px;
                border: 1px solid #ccc;
                font-size: 16px;
                margin-top: 5px;
            }
            .year-btn {
                padding: 10px 15px;
                background-color: #e0e0e0;
                border: 1px solid #ccc;
                cursor: pointer;
                border-radius: 4px;
            }
            .year-btn:hover {
                background-color: #d0d0d0;
            }
            .year-btn.active {
                background-color: #0078D7;
                color: white;
            }
            label {
                font-weight: bold;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <div class="header">
                <h1>India Sea/Marine Water Quality Map</h1>
            </div>
            <div class="selectors">
                <div class="parameter-selector">
                    <label for="parameter">Select Parameter:</label>
                    <select id="parameter" onchange="changeParameter()">
                        <option value="pH">pH</option>
                        <option value="DO">Dissolved Oxygen (DO)</option>
                        <option value="BOD">Biochemical Oxygen Demand (BOD)</option>
                    </select>
                </div>
                <div class="year-selector">
    """
    
    # Add year buttons
    for year in range(2012, 2024):
        active = 'active' if year == 2012 else ''
        html_content += f'<button class="year-btn {active}" onclick="changeYear({year})">{year}</button>\n'
    
    html_content += """
                </div>
            </div>
            <div class="content">
                <div class="map-container">
                    <iframe id="map-frame" src="maps/sea_pH/india_sea_ph_map_2012.html"></iframe>
                </div>
            </div>
        </div>
        <script>
            let currentYear = 2012;
            let currentParameter = 'pH';
            
            function updateMap() {
                // Update iframe source based on current selections
                const paramFolder = 'sea_' + currentParameter;
                const paramLower = currentParameter.toLowerCase();
                document.getElementById('map-frame').src = `maps/${paramFolder}/india_sea_${paramLower}_map_${currentYear}.html`;
            }
            
            function changeParameter() {
                currentParameter = document.getElementById('parameter').value;
                updateMap();
            }
            
            function changeYear(year) {
                currentYear = year;
                updateMap();
                
                // Update active button
                const buttons = document.querySelectorAll('.year-btn');
                buttons.forEach(btn => {
                    if (btn.textContent == year) {
                        btn.classList.add('active');
                    } else {
                        btn.classList.remove('active');
                    }
                });
            }
        </script>
    </body>
    </html>
    """
    
    # Write HTML to file
    with open('india_sea_water_quality_map_selector.html', 'w') as f:
        f.write(html_content)
    
    print("Sea water parameter and year selector app created! Open 'india_sea_water_quality_map_selector.html' in your browser.")

# Run the parameter and year selector app
create_parameter_year_selector_app()

Generated sea water pH map for 2012
Generated sea water DO map for 2012
Generated sea water BOD map for 2012
Generated sea water pH map for 2013
Generated sea water DO map for 2013
Generated sea water BOD map for 2013
Generated sea water pH map for 2014
Generated sea water DO map for 2014
Generated sea water BOD map for 2014
Generated sea water pH map for 2015
Generated sea water DO map for 2015
Generated sea water BOD map for 2015
Generated sea water pH map for 2016
Generated sea water DO map for 2016
Generated sea water BOD map for 2016
Generated sea water pH map for 2017
Generated sea water DO map for 2017
Generated sea water BOD map for 2017
Generated sea water pH map for 2018
Generated sea water DO map for 2018
Generated sea water BOD map for 2018
Generated sea water pH map for 2019
Generated sea water DO map for 2019
Generated sea water BOD map for 2019
Generated sea water pH map for 2020
Generated sea water DO map for 2020
Generated sea water BOD map for 2020
Generated sea water